In [1]:
!pip install nltk
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 5.2 MB/s eta 0:00:00a 0:00:01


In [2]:
import nltk
nltk.download('punkt')

from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to /home/student/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
def tokenize(sentence):
  return nltk.word_tokenize(sentence)

def stem(word):
  return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):
   tokenized_sentence=[stem(w) for w in tokenized_sentence]
   bag=np.zeros(len(all_words), dtype=np.float32)
   for idx, w in enumerate(all_words):
    if w in tokenized_sentence:
      bag[idx]=1.0
   return bag

# a=['hello','how','are','you']
# words=['hi','hello','how','you','bye','thank']
# bog=bag_of_words(a,words)
# print(bog)


# a="How are you"
# print(a)
# a=tokenize(a)
# print(a)

In [4]:
# words=["organize","organizes","organizing"]
# stemmed_words=[stem(w)for w in words]
# print(stemmed_words)

In [5]:
import json

In [5]:
from google.colab import files
uploaded=files.upload()


Saving intents.json to intents.json


In [6]:
import json
with open('intents.json','r') as i:
  intents=json.load(i)
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hello, thanks for visiting', 'Good to see you again', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'hours', 'patterns': ['What hours are you open?', 'What are your hours?', 'When are you open?'], 'responses': ["We're open every day 9am-9pm", 'Our hours are 9am-9pm every day']}, {'tag': 'payments', 'patterns': ['Do you take online payments?', 'Are you cash only?', 'What are the means of payment?'], 'responses': ['We accept payment via khalti', 'We accept cash as well as online payment']}, {'tag': 'open_today', 'patterns': ['Are you open today?', 'When 

In [7]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [8]:
class NeuralNet(nn.Module):
  def __init__(self,input_size,hidden_size,num_classes):
    super(NeuralNet,self).__init__()
    self.l1=nn.Linear(input_size,hidden_size)
    self.l2=nn.Linear(hidden_size,hidden_size)
    self.l3=nn.Linear(hidden_size,num_classes)
    self.relu=nn.ReLU()
  def forward(self,x):
    out=self.l1(x)
    out=self.relu(out)
    out=self.l2(out)
    out=self.relu(out)
    out=self.l3(out)
    return out


In [9]:
# !pip install cuda

In [10]:
all_words=[]
tags=[]
xy=[]
for intent in intents['intents']:
  tag=intent['tag']
  tags.append(tag)
  for pattern in intent['patterns']:
    w=tokenize(pattern)
    all_words.extend(w)
    xy.append((w,tag))
ignore_words=['?','.','!','.']
all_words=[stem(w) for w in all_words if w not in ignore_words]
# print(all_words)

X_train=[]
Y_train=[]
for (pattern_sentence,tag) in xy:
  bag=bag_of_words(pattern_sentence,all_words)
  X_train.append(bag)

  label=tags.index(tag)
  Y_train.append(label)
X_train=np.array(X_train)
Y_train=np.array(Y_train)
# print(X_train)
# print(Y_train)


class ChatDataset(Dataset):
  def __init__(self):
    self.n_samples=len(X_train)
    self.x_data=X_train
    self.y_data=Y_train


  def __getitem__(self,index):
    return self.x_data[index],self.y_data[index]

  def __len__(self):
    return self.n_samples

batch_size=8
hidden_size=8
output_size=len(tags)
input_size=len(X_train[0])
learning_rate=0.001
num_epochs=1000
# print(input_size,len(all_words))
# print(output_size,tags)
dataset=ChatDataset()
train_loader=DataLoader(dataset=dataset, batch_size=batch_size,shuffle=True,num_workers=2)


device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=NeuralNet(input_size,hidden_size,output_size).to(device)

criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

for epoch in range(num_epochs):
  for(words,labels) in train_loader:
    words=words.to(device)
    labels=labels.to(device)

    outputs=model(words)
    loss=criterion(outputs,labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if(epoch+1)%100==0:
      print(f'epoch (epoch+1)/{num_epochs}, loss={loss.item():.4f}')

print(f'final loss, loss={loss.item():.4f}')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')










epoch (epoch+1)/1000, loss=0.1210
epoch (epoch+1)/1000, loss=0.1970
epoch (epoch+1)/1000, loss=0.5216
epoch (epoch+1)/1000, loss=0.5649
epoch (epoch+1)/1000, loss=0.4903
epoch (epoch+1)/1000, loss=0.4601
epoch (epoch+1)/1000, loss=0.5253
epoch (epoch+1)/1000, loss=0.1527
epoch (epoch+1)/1000, loss=0.4125
epoch (epoch+1)/1000, loss=0.5199
epoch (epoch+1)/1000, loss=0.0887
epoch (epoch+1)/1000, loss=0.0634
epoch (epoch+1)/1000, loss=0.1354
epoch (epoch+1)/1000, loss=0.0175
epoch (epoch+1)/1000, loss=0.0478
epoch (epoch+1)/1000, loss=0.1585
epoch (epoch+1)/1000, loss=0.1822
epoch (epoch+1)/1000, loss=0.0909
epoch (epoch+1)/1000, loss=0.0981
epoch (epoch+1)/1000, loss=0.0237
epoch (epoch+1)/1000, loss=0.0317
epoch (epoch+1)/1000, loss=0.0370
epoch (epoch+1)/1000, loss=0.1022
epoch (epoch+1)/1000, loss=0.0145
epoch (epoch+1)/1000, loss=0.0068
epoch (epoch+1)/1000, loss=0.0031
epoch (epoch+1)/1000, loss=0.0145
epoch (epoch+1)/1000, loss=0.0034
epoch (epoch+1)/1000, loss=0.0069
epoch (epoch+1

In [11]:
import random

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

def get_response(msg):
    sentence = tokenize(msg)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.5:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                return random.choice(intent['responses'])

    return "I do not understand..."


if __name__ == "__main__":
    print("Let's chat! (type 'quit' to exit)")
    while True:
        sentence = input("You: ")
        if sentence == "quit":
            break

        resp = get_response(sentence)
        print(resp)

Let's chat! (type 'quit' to exit)
You: Hello
Hello, thanks for visiting
You: Book an appointment
I do not understand...
You: Measures for payment
Have a nice day
You: What are the means of payment
We accept cash as well as online payment
You: exit
Have a nice day
You: quit


In [12]:
get_response('write blog')

'Blog section is for writing original articles and contents to share to the public.'

In [13]:
get_response('What are the means of payment')

'We accept cash as well as online payment'